In [1]:
import pandas as pd

In [2]:
chunk_size = 100000
chunks = pd.read_csv("biobank/hesin_diag.txt", sep="\t", chunksize=chunk_size)

In [3]:
myocardinal_diag_patients_IC21_lst = []
for chunk in chunks:
    myocardinal_diag_patients_IC21 = chunk[chunk['diag_icd10'].str.contains("I21", na=False)]
    myocardinal_diag_patients_IC21_lst.append(myocardinal_diag_patients_IC21)



In [4]:

all_patients_with_myocardinal_diag_df = pd.concat(myocardinal_diag_patients_IC21_lst, ignore_index=True)
all_patients_with_myocardinal_diag_df

,eid,ins_index,arr_index,level,diag_icd9,diag_icd9_nb,diag_icd10,diag_icd10_nb
0,1000013,1,0,1,NaN,NaN,I211,NaN
1,1000048,11,0,1,NaN,NaN,I214,NaN
2,1000238,2,0,1,NaN,NaN,I211,NaN
3,1000238,1,0,1,NaN,NaN,I211,NaN
4,1000484,13,1,2,NaN,NaN,I219,NaN
...,...,...,...,...,...,...,...,...
39303,6022345,3,0,1,NaN,NaN,I214,NaN
39304,6022448,14,0,1,NaN,NaN,I219,NaN
39305,6023001,8,0,1,NaN,NaN,I212,NaN
39306,6023562,3,0,1,NaN,NaN,I219,NaN


In [5]:
def calculate_age_at_mi(row):
    age_at_recruitment = row['21022']  # Age at recruitment
    date_of_recruitment = pd.to_datetime(row['recruitment_date'])  # Assuming you have a column with the recruitment date
    date_of_mi = pd.to_datetime(row['131298'])  # Using the correct field for date of MI
    
    age_at_mi = age_at_recruitment + ((date_of_mi - date_of_recruitment).days / 365.25)
    return age_at_mi

In [6]:
def process_biobank_file(file_path, mi_eids_df):
    chunksize = 10000  # Adjust based on your memory capacity
    results = []

    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        # Merge chunk with mi_eids_df to filter for relevant rows
        mi_chunk = pd.merge(chunk, mi_eids_df, on='eid', how='inner')
        
        if not mi_chunk.empty:
            mi_chunk['age_at_mi'] = mi_chunk.apply(calculate_age_at_mi, axis=1)
            results.append(mi_chunk[['eid', '21022', '131298', 'age_at_mi']])

    # Concatenate all the results
    result_df = pd.concat(results, ignore_index=True)
    return result_df

In [ ]:
biobank_file_path = 'biobank/ukb672220.csv'  # Replace with your actual file path

# Process the biobank file and get the results
mi_results = process_biobank_file(biobank_file_path, all_patients_with_myocardinal_diag_df)